背包最多裝150公斤的物品, 現在有7個物品


物品 |1 |2| 3 |4 |5| 6| 7 
-----|----|---|---|----|-----|----
價值 |10 |40| 30| 50 |35| 40 |30 
重量| 35 |30 |60| 50 |40| 10| 25

基因編碼 (chromosome representation) 

使用2進位編碼來表示每一個基因 

物品相對索引值 0代表不選此物品, 1代表選擇此物品

適應函數 (fitness function) 

對背包內裝入物品的價值進行評估, 取價值最大的那個 

裝入背包內的物品總重量不能超過背包容量 

而對非法狀態列出懲罰, 將其適應度設成5



In [10]:
import random
import numpy as np
import pandas as pd

article = {1: [10, 35], 2: [40, 30], 3: [30, 60], 4: [50, 50], 5: [35, 40], 6: [40, 10], 7: [30, 25]}

population_size = 10  #族群大小
selection_rate = 0.5 #選擇率
crossover_rate = 0.8  #交配率
mutation_rate = 0.6  #突變率
epoch = 100

def fitness_function(chromosome=None, max_weight=150, data=None, penalty_value=5):
    """輸入一組染色體求適應值"""
    fitness_value = 0 #適應值 
    weight = 0        #重量
    for i, c in enumerate(chromosome):
        if chromosome[i]:
            fitness_value += data[i+1][0]
            weight += data[i+1][1]   
    if weight >max_weight:           #懲罰
        fitness_value = penalty_value
    return fitness_value

def Roulette(data=None):
    """輸入族群給出輪盤選中的個體"""
    import random
    import numpy as np
    r = random.uniform(0, 1)
    for i in range(len(np.array(data))):
        if sum((np.array(data)/sum(np.array(data)))[0:i+1]) > r:
            break
    return i

def crossover(chromosome_1, chromosome_2, point=2):
    """輸入兩組染色體做交配"""
    p = random.sample(range(len(chromosome_1)), k=point)
    new_chromosome_1 = []
    new_chromosome_2 = []
    for i in range(len(chromosome_1)):
        if i in p:
            new_chromosome_1.append(chromosome_2[i])
            new_chromosome_2.append(chromosome_1[i])
        else:
            new_chromosome_1.append(chromosome_1[i])
            new_chromosome_2.append(chromosome_2[i])
            
    return new_chromosome_1, new_chromosome_2

def mutation(chromosome, point=1):
    """輸入一組染色體做突變"""
    p = random.sample(range(len(chromosome)), k=point)
    new_chromosome = chromosome
    for i in p:
        if chromosome[i] == 0:
            new_chromosome[i] = 1
        else:
            new_chromosome[i] = 0
    return new_chromosome

chromosome_0 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_1 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_2 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_3 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_4 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_5 = np.array([random.randint(0, 1) for _ in range(population_size)])
chromosome_6 = np.array([random.randint(0, 1) for _ in range(population_size)])

population = np.stack((chromosome_0, chromosome_1, chromosome_2, chromosome_3, chromosome_4, chromosome_5,  chromosome_6), axis=-1)
population  #初始族群

array([[1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 1]])

#### 演化

In [11]:
counter = 0
while counter<epoch:
    value = []
    for p in population:
        value.append(fitness_function(chromosome=p, max_weight=150, data=article, penalty_value=5))
    value = np.array(value)    #適應值

    dict = {"chromosome_0":population[:,0], 
            "chromosome_1":population[:,1],
            "chromosome_2":population[:,2],
            "chromosome_3":population[:,3],
            "chromosome_4":population[:,4],
            "chromosome_5":population[:,5],
            "chromosome_6":population[:,6],
            "fitness value":value}
    df = pd.DataFrame(dict)
    sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序

    population_selection = np.stack((sort_df["chromosome_0"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_1"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_2"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_3"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_4"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_5"].values[:int(len(sort_df)*selection_rate)],  
                                     sort_df["chromosome_6"].values[:int(len(sort_df)*selection_rate)]), axis=-1)

    population_crossover = population_selection.copy()  #直接複製

    c1type = False
    c2type = False
    while len(population_crossover) < population_size:
        for i, c in enumerate(population_selection):
            if crossover_rate > random.uniform(0, 1):
                if c1type:
                    c2 = population_selection[i]
                    c2type = True
                else:
                    c1 = population_selection[i]
                    c1type = True

                if c1type and c2type:
                    new_c1, new_c2 = crossover(c1, c2, point=2)
                    population_crossover = np.append(population_crossover,[new_c1],axis=0)
                    if len(population_crossover) >= population_size:
                        break
                    population_crossover = np.append(population_crossover,[new_c2],axis=0)
                    c1type = False
                    c2type = False
                    if len(population_crossover) >= population_size:
                        break
                else:
                    pass
        if len(population_crossover) >= population_size:
            break
    population_mutation = population_crossover.copy()

    for i, c in enumerate(population_mutation[int(population_size*selection_rate):], int(population_size*selection_rate)):  
        if mutation_rate > random.uniform(0, 1):
            population_mutation[i] = mutation(c, point=1)

    population = population_mutation.copy()
    counter += 1
    print(counter,max(value))

1 155
2 155
3 155
4 170
5 170
6 170
7 170
8 170
9 170
10 170
11 170
12 170
13 170
14 170
15 170
16 170
17 170
18 170
19 170
20 170
21 170
22 170
23 170
24 170
25 170
26 170
27 170
28 170
29 170
30 170
31 170
32 170
33 170
34 170
35 170
36 170
37 170
38 170
39 170
40 170
41 170
42 170
43 170
44 170
45 170
46 170
47 170
48 170
49 170
50 170
51 170
52 170
53 170
54 170
55 170
56 170
57 170
58 170
59 170
60 170
61 170
62 170
63 170
64 170
65 170
66 170
67 170
68 170
69 170
70 170
71 170
72 170
73 170
74 170
75 170
76 170
77 170
78 170
79 170
80 170
81 170
82 170
83 170
84 170
85 170
86 170
87 170
88 170
89 170
90 170
91 170
92 170
93 170
94 170
95 170
96 170
97 170
98 170
99 170
100 170


In [12]:
value = []
for p in population:
    value.append(fitness_function(chromosome=p, max_weight=150, data=article, penalty_value=5))
value = np.array(value)    #適應值

dict = {"chromosome_0":population[:,0], 
        "chromosome_1":population[:,1],
        "chromosome_2":population[:,2],
        "chromosome_3":population[:,3],
        "chromosome_4":population[:,4],
        "chromosome_5":population[:,5],
        "chromosome_6":population[:,6],
        "fitness value":value}
df = pd.DataFrame(dict)
sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序
sort_df

,chromosome_0,chromosome_1,chromosome_2,chromosome_3,chromosome_4,chromosome_5,chromosome_6,fitness value
0,1,1,0,1,0,1,1,170
1,1,1,0,1,0,1,1,170
2,1,1,0,1,0,1,1,170
3,1,1,0,1,0,1,1,170
4,1,1,0,1,0,1,1,170
5,1,1,0,1,0,1,1,170
6,1,1,0,1,0,1,1,170
7,1,1,0,1,0,1,1,170
9,1,1,0,1,0,1,0,140
8,1,0,0,1,0,1,1,130
